# SpaceX Launch Sites Location Visualization (Folium)

**Dataset Source:** [spacex_launch_geo.csv](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv)

## Task 1: Mark all launch sites on a map

In [4]:
import folium
import pandas as pd

# Load CSV directly from URL
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
df = pd.read_csv(url)

# Create a folium map
launch_map = folium.Map(location=[28.5, -80.6], zoom_start=5)

# Add launch site markers
for _, row in df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site'],
        icon=folium.Icon(color='blue')
    ).add_to(launch_map)

launch_map

### Task 1: Map Analysis Summary

- **Are all launch sites in proximity to the Equator line?**  
  Not all launch sites are very close to the Equator. While some launch sites like CCAFS SLC 40 and KSC LC 39A (in Florida) are relatively near the Equator, others like VAFB SLC 4E (in California) are located farther north.

- **Are all launch sites in very close proximity to the coast?**  
  Yes, all launch sites are located near coastlines. This is likely for safety reasons — launching over the ocean reduces the risk to human life and infrastructure in case of a launch failure or debris falling.

**Conclusion:**  
Most launch sites are near the coast, which is a strategic choice for safety. Their distance from the Equator varies depending on the launch mission and orbital requirements.

## Task 2: Mark the success/failed launches for each site on the map

In [2]:
# Add color-coded markers based on class (success/failure)
launch_map2 = folium.Map(location=[28.5, -80.6], zoom_start=5)

for _, row in df.iterrows():
    color = 'green' if row['class'] == 1 else 'red'
    folium.CircleMarker(
        location=[row['Lat'], row['Long']],
        radius=5,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"{row['Launch Site']} - {'Success' if row['class'] == 1 else 'Failure'}"
    ).add_to(launch_map2)

launch_map2

## Task 3: Calculate the distances between a launch site to its proximities

In [5]:
import folium
from math import sin, cos, sqrt, atan2, radians

# Example launch site (CCAFS LC-40)
launch_lat = 28.562302
launch_lon = -80.577356

# Example closest coastline point (manually measured using MousePosition)
coast_lat = 28.56367
coast_lon = -80.57163

# Function to calculate distance using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # Radius of the Earth in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Calculate distance
distance_km = calculate_distance(launch_lat, launch_lon, coast_lat, coast_lon)

# Create map
site_map = folium.Map(location=[launch_lat, launch_lon], zoom_start=12)

# Add marker for launch site
folium.Marker(
    location=[launch_lat, launch_lon],
    popup='Launch Site: CCAFS LC-40',
    icon=folium.Icon(color='blue')
).add_to(site_map)

# Add marker for coastline point with distance display
folium.Marker(
    location=[coast_lat, coast_lon],
    icon=folium.DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color:#d35400;"><b>{distance_km:.2f} KM</b></div>',
    )
).add_to(site_map)

# Draw a line between launch site and coast
folium.PolyLine(
    locations=[[launch_lat, launch_lon], [coast_lat, coast_lon]],
    color='red',
    weight=2
).add_to(site_map)

# Show map
site_map